In [217]:


import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [218]:
#load csv

customers = pd.read_csv("Customers.csv")
churn = pd.read_csv("Churn.csv")
subscriptions = pd.read_csv("Subscriptions.csv")
transactions = pd.read_csv("Transactions.csv")

In [219]:
#merge: cust left join subscription

df = customers.merge(subscriptions, on = "CustomerID", how = "left")


In [193]:
df.head(4)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType
0,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,1.0,10-01-2022,09-01-2023,Annual
1,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,44.0,10-01-2023,09-01-2024,Monthly
2,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,101.0,01-07-2023,30-06-2024,Monthly
3,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,2.0,15-12-2021,14-12-2022,Monthly


In [194]:
transactions.head(3)

,TransactionID,CustomerID,TransactionDate,Amount,TransactionType
0,1,15,01-01-2024,32,Purchase
1,2,22,02-01-2024,52,Refund
2,3,43,03-01-2024,165,Purchase


In [220]:
#Prepare tranasaction summary for customer
#spend logic => for each cust Sum(purchase)-Sum(refund)

transactions["AdjustedAmount"] = transactions.apply(lambda row: -row["Amount"] if row["TransactionType"] == "Refund" else row["Amount"], axis=1)

# THis also works::: transactions["test"] = np.where(transactions["TransactionType"] == "Refund", -transactions["Amount"], transactions["Amount"])
transactions.head(3)


,TransactionID,CustomerID,TransactionDate,Amount,TransactionType,AdjustedAmount
0,1,15,01-01-2024,32,Purchase,32
1,2,22,02-01-2024,52,Refund,-52
2,3,43,03-01-2024,165,Purchase,165


In [221]:
transaction_summary =  transactions.groupby("CustomerID")["AdjustedAmount"].sum().reset_index().rename(columns = {"AdjustedAmount":"CustomerLifetime"})

In [197]:
transaction_summary.head(20)

,CustomerID,CustomerLifetime
0,11,195
1,15,32
2,17,-73
3,19,-140
4,21,-32
5,22,-52
6,23,196
7,25,-122
8,26,-20
9,28,-153


In [222]:
#merge the transaction summary with the df "the 1st merger"

df = df.merge(transaction_summary, on = "CustomerID", how = "left")

In [223]:
df.head(5)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime
0,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,1.0,10-01-2022,09-01-2023,Annual,NaN
1,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,44.0,10-01-2023,09-01-2024,Monthly,NaN
2,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,101.0,01-07-2023,30-06-2024,Monthly,NaN
3,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,2.0,15-12-2021,14-12-2022,Monthly,NaN
4,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,45.0,15-12-2022,14-12-2023,Annual,NaN


In [224]:
df["CustomerLifetime"].describe()

count    262.000000
mean      -8.629771
std      126.855170
min     -232.000000
25%     -122.000000
50%      -37.000000
75%      117.000000
max      200.000000
Name: CustomerLifetime, dtype: float64

In [201]:
df["CustomerLifetime"].value_counts()

CustomerLifetime
 143.0    5
-156.0    5
 165.0    5
-66.0     4
 200.0    4
         ..
-34.0     1
 104.0    1
-136.0    1
 176.0    1
 29.0     1
Name: count, Length: 170, dtype: int64

In [202]:
df["CustomerLifetime"].unique()

array([  nan,  195.,   32.,  -73., -140.,  -32.,  -52.,  196., -122.,
        -20., -153.,  200.,  -66., -156.,  158., -145.,  143.,  120.,
       -200.,  165.,  -69.,  -99., -143.,  142., -150.,  159.,  -81.,
       -170.,  157.,  -68.,  145.,  -93.,  198., -141.,  -35.,  -44.,
         87., -111., -149.,   73., -102., -120.,  140.,  134.,   61.,
       -118.,  171., -199.,   96.,  -23., -116.,  185., -133., -139.,
       -176., -107.,   98.,   94., -124.,  -70.,  -46.,  -61.,   24.,
        103.,   43.,  131., -119.,  189., -174., -178., -162.,   23.,
         21.,  155.,   52.,  141.,   47., -168.,   63.,  -43.,   83.,
       -100.,  -72.,  108., -125.,   57.,  -60.,  -78.,  -38.,   22.,
       -232.,   41., -195.,  -71.,  153.,  -40., -105., -186.,   74.,
        128.,   76.,  181.,  -51.,   75.,  -47., -115., -147.,   28.,
        118., -181.,  -28., -185., -192., -177., -189.,  191., -123.,
         89.,  -48., -106.,  -55.,   35., -180.,   37.,  169., -171.,
       -103.,   62.,

In [203]:
churn.head(20)

,ChurnID,CustomerID,ChurnDate,Reason
0,1,22,02-01-2024,Poor Customer Service
1,2,56,06-06-2024,High Prices
2,3,45,10-10-2024,Product Quality Issues
3,4,54,12-12-2024,Lack of Product Features
4,5,82,14-02-2024,Competitor Offerings
5,6,67,15-01-2024,Inconvenience
6,7,17,21-01-2024,Lack of Engagement
7,8,33,18-01-2024,Unmet Expectations
8,9,91,20-01-2024,Price Increases
9,10,23,23-01-2024,Personal Circumstances


In [225]:
df = df.merge(churn[["CustomerID","ChurnDate"]],on = "CustomerID" , how = "left")

In [205]:
df.head(100)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime,ChurnDate
0,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,1.0,10-01-2022,09-01-2023,Annual,NaN,NaN
1,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,44.0,10-01-2023,09-01-2024,Monthly,NaN,NaN
2,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,101.0,01-07-2023,30-06-2024,Monthly,NaN,NaN
3,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,2.0,15-12-2021,14-12-2022,Monthly,NaN,NaN
4,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,45.0,15-12-2022,14-12-2023,Annual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,71.0,22-02-2022,21-02-2023,Annual,200.0,24-01-2024
96,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024
97,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024
98,30,Brian,Gonzalez,brian.g@example.com,890-123-4567,12-06-2020,Inactive,Asia,30.0,12-06-2020,11-06-2021,Annual,NaN,30-01-2024


In [226]:
#clean the data and handle the values like NaN, etc. becasue the regression works on Numeric data

df["Churned"] = df["ChurnDate"].notnull().astype(int)

In [207]:
df.head(100)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime,ChurnDate,Churned
0,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,1.0,10-01-2022,09-01-2023,Annual,NaN,NaN,0
1,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,44.0,10-01-2023,09-01-2024,Monthly,NaN,NaN,0
2,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,101.0,01-07-2023,30-06-2024,Monthly,NaN,NaN,0
3,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,2.0,15-12-2021,14-12-2022,Monthly,NaN,NaN,0
4,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,45.0,15-12-2022,14-12-2023,Annual,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,71.0,22-02-2022,21-02-2023,Annual,200.0,24-01-2024,1
96,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024,1
97,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024,1
98,30,Brian,Gonzalez,brian.g@example.com,890-123-4567,12-06-2020,Inactive,Asia,30.0,12-06-2020,11-06-2021,Annual,NaN,30-01-2024,1


In [227]:
#Clean the NaN from customerlifetime

df["CustomerLifetime"] = df["CustomerLifetime"].fillna(0)

In [209]:
df.head(100)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime,ChurnDate,Churned
0,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,1.0,10-01-2022,09-01-2023,Annual,0.0,NaN,0
1,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,44.0,10-01-2023,09-01-2024,Monthly,0.0,NaN,0
2,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,101.0,01-07-2023,30-06-2024,Monthly,0.0,NaN,0
3,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,2.0,15-12-2021,14-12-2022,Monthly,0.0,NaN,0
4,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,45.0,15-12-2022,14-12-2023,Annual,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,71.0,22-02-2022,21-02-2023,Annual,200.0,24-01-2024,1
96,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024,1
97,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024,1
98,30,Brian,Gonzalez,brian.g@example.com,890-123-4567,12-06-2020,Inactive,Asia,30.0,12-06-2020,11-06-2021,Annual,0.0,30-01-2024,1


In [228]:
df["IsInactive"] = (df["Status"] == "Inactive").astype(int)

In [211]:
df.head(100)

,CustomerID,FirstName,LastName,Email,PhoneNumber,JoinDate,Status,Region,SubscriptionID,StartDate,EndDate,PlanType,CustomerLifetime,ChurnDate,Churned,IsInactive
0,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,1.0,10-01-2022,09-01-2023,Annual,0.0,NaN,0,0
1,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,44.0,10-01-2023,09-01-2024,Monthly,0.0,NaN,0,0
2,1,John,Doe,john.doe@example.com,123-456-7890,10-01-2022,Active,North America,101.0,01-07-2023,30-06-2024,Monthly,0.0,NaN,0,0
3,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,2.0,15-12-2021,14-12-2022,Monthly,0.0,NaN,0,1
4,2,Jane,Smith,jane.smith@example.com,098-765-4321,15-12-2021,Inactive,Europe,45.0,15-12-2022,14-12-2023,Annual,0.0,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,71.0,22-02-2022,21-02-2023,Annual,200.0,24-01-2024,1,0
96,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024,1,0
97,29,Abby,Flores,abby.f@example.com,789-012-3456,22-02-2021,Active,Europe,129.0,15-07-2023,14-07-2024,Monthly,200.0,24-01-2024,1,0
98,30,Brian,Gonzalez,brian.g@example.com,890-123-4567,12-06-2020,Inactive,Asia,30.0,12-06-2020,11-06-2021,Annual,0.0,30-01-2024,1,1


In [229]:
#define X and Y

X = df[["CustomerLifetime","IsInactive"]]
y = df["Churned"]

In [230]:
#Improvement Section: Scale the numerical data

scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)

In [236]:
#split data for training & testing

X_train, X_test, y_train, y_test = train_test_split(X_scaler,y,test_size = 0.2, random_state = 42)

In [237]:
#Train the logistic regression model in the training data.

model = LogisticRegression(max_iter = 1000, C=2)
model.fit(X_train,y_train)

LogisticRegression(C=2, max_iter=1000)

In [238]:
#Use the model on splited test to predict

y_pred = model.predict(X_test)

In [239]:
#Evalutate the Model

print("Model Evaluation\n")

print("Accuracy:", round(accuracy_score(y_test,y_pred),2))
print("Precision:", precision_score(y_test,y_pred))
print("Recall:", recall_score(y_test,y_pred))
print("F1 Score:", f1_score(y_test,y_pred))
print("--------------------------------------")
print("Confusion Matrix:", confusion_matrix(y_test,y_pred))

Model Evaluation

Accuracy: 0.65
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
--------------------------------------
Confusion Matrix: [[57  0]
 [31  0]]


C:\Users\anike\anaconda_\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [184]:
#predict on new data:
new_data = pd.read_csv("Sample_Test_Data.csv")
new_data.head(4)


,CustomerLifetime,IsInactive
0,102,1
1,435,1
2,860,1
3,270,1


In [185]:
#predict using the loaded model for the new data
#X_new = new_data[["CustomerLifetime","IsInactive"]]

new_data["Predicted_Churn"] = model.predict(new_data)
new_data.head(20)

,CustomerLifetime,IsInactive,Predicted_Churn
0,102,1,0
1,435,1,0
2,860,1,0
3,270,1,0
4,106,1,0
5,71,1,0
6,700,0,0
7,20,1,0
8,614,0,0
9,121,1,0


In [186]:
new_data.to_csv("churn_prediction_output.csv",index = False)